In [ ]:
import pandas as pd 
from transformers import AutoTokenizer 
df = pd.read_csv('../data/raw/clothing_reviews.csv')
df = df[['Review Text', 'Recommended IND']].dropna() 


model_name = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)



sample = 'This dress is amazing!'


tokens = tokenizer(sample, padding= 'max_length', truncation = True, max_length=10)

print(f"원문: {sample}")
print(f"컴퓨터가 보는 숫자들: {tokens['input_ids']}")

/opt/homebrew/anaconda3/envs/sentiment_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


원문: This dress is amazing!
컴퓨터가 보는 숫자들: [101, 2023, 4377, 2003, 6429, 999, 102, 0, 0, 0]


In [3]:
import torch 
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(device)

mps


In [6]:
from torch.utils.data import Dataset 

class ReviewDataset(Dataset):
    def __init__(self, reviews, labels, tokenizer, max_len):
        self.reviews = reviews
        self.labels = labels 
        self.tokenizer = tokenizer
        self.max_len = max_len
    def __len__(self):
        return len(self.reviews)
    def __getitem__(self, item):
        review = str(self.reviews[item])
        label = self.labels[item]

        encoding = self.tokenizer(
            review,
            add_special_tokens= True, 
            max_length  = self.max_len,
            return_token_type_ids = False,
            padding='max_length',
            truncation = True,
            return_tensors='pt',
        )
        return {
            'review_text': review,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [ ]:
import os
os.makedirs('../data/processed', exist_ok=True)

print(df.head())

df.to_csv('../data/processed/cleaned_reviews.csv', index=False)

print("데이터가 'data/processed/cleaned_reviews.csv'에저장되었습니다!")